In [ ]:
!pip install transformers accelerate av torch numpy -q

In [ ]:
import av
import torch
import numpy as np
from transformers import VideoLlavaForConditionalGeneration, VideoLlavaProcessor

In [ ]:
def load_model_and_processor():
    model = VideoLlavaForConditionalGeneration.from_pretrained(
        "LanguageBind/Video-LLaVA-7B-hf",
        torch_dtype=torch.float16,
        device_map="auto",
        offload_folder="offload",
        offload_buffers=True
    )

    # Optional: Use torch.compile (يمكنك إزالته لو عمل مشاكل)
    model = torch.compile(model)

    processor = VideoLlavaProcessor.from_pretrained("LanguageBind/Video-LLaVA-7B-hf")

    return model, processor

In [ ]:
def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

def analyze_injury(video_path, model, processor):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    container = av.open(video_path)
    total_frames = container.streams.video[0].frames
    indices = np.linspace(0, total_frames - 1, num=4, dtype=int)
    video = read_video_pyav(container, indices)

    prompt = """USER: <video>
You are an expert AI analyst reviewing a football match injury video.

Please watch the clip carefully and extract detailed information about the injury.

🔍 Your task is to analyze the video and return the information in a structured table format with the following columns:

| Injury Description | Injury Severity | Medical Team Involved | Was Play Stopped? | Injury Location | Did the Player Leave the Field? |
|--------------------|------------------|------------------------|--------------------|------------------|-------------------------------|

🔸 Column details:
- **Injury Description**: Briefly describe what happened in 2–3 sentences (e.g., "The player fell after a heavy tackle from the opponent and showed signs of pain in his left knee.").
- **Injury Severity**: Choose from [Minor, Moderate, Severe].
- **Medical Team Involved**: Yes / No — Did the medical team enter the field?
- **Was Play Stopped?**: Yes / No — Was the match interrupted due to the injury?
- **Injury Location**: Be specific (e.g., right ankle, left shoulder, head).
- **Did the Player Leave the Field?**: Yes / No — Did the player continue playing or leave?

📌 Return only the completed table in your response.

--- Example Output:

| Injury Description                                               | Injury Severity | Medical Team Involved | Was Play Stopped? | Injury Location | Did the Player Leave the Field? |
|------------------------------------------------------------------|------------------|------------------------|--------------------|------------------|-------------------------------|
| The player collapsed after a strong challenge on his right leg. He was visibly in pain and held his shin for several seconds. The medical team quickly came to assess the injury. | Moderate         | Yes                    | Yes                | Right shin       | No                            |

ASSISTANT:"""

    inputs = processor(text=prompt, videos=video, return_tensors="pt").to(device)
    out = model.generate(**inputs, max_new_tokens=250)
    result = processor.batch_decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    return result

In [ ]:
video_path = "/content/drive/MyDrive/Ai League/injury.mp4"

# حمل النموذج مرة واحدة فقط
model, processor = load_model_and_processor()



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# استخدم النموذج لتحليل أي فيديو بدون تحميله مرة أخرى
result = analyze_injury(video_path, model, processor)

print(result)

Using device: cuda
['USER: \nYou are an expert AI analyst reviewing a football match injury video.\n\nPlease watch the clip carefully and extract detailed information about the injury.\n\n🔍 Your task is to analyze the video and return the information in a structured table format with the following columns:\n\n| Injury Description | Injury Severity | Medical Team Involved | Was Play Stopped? | Injury Location | Did the Player Leave the Field? |\n|--------------------|------------------|------------------------|--------------------|------------------|-------------------------------|\n\n🔸 Column details:\n- **Injury Description**: Briefly describe what happened in 2–3 sentences (e.g., "The player fell after a heavy tackle from the opponent and showed signs of pain in his left knee.").\n- **Injury Severity**: Choose from [Minor, Moderate, Severe].\n- **Medical Team Involved**: Yes / No — Did the medical team enter the field?\n- **Was Play Stopped?**: Yes / No — Was the match interrupted du

In [ ]:
import re

def extract_model_response(result):
    """
    Extracts the model's response, specifically the table data, from the output string.

    Args:
        result (List[str]): The output from processor.batch_decode, e.g., [output_text]

    Returns:
        str: The extracted model's table response
    """
    if isinstance(result, list):
        output_text = result[0]
    else:
        output_text = result

    # Extract the part of the response that contains the table data
    match = re.search(r"ASSISTANT:\s*(\|.*\|)", output_text, re.DOTALL)

    if match:
        return match.group(1).strip()
    else:
        print("⚠️ Unable to extract the model's table response.")
        return None

# Example usage
model_response = extract_model_response(result)
print(model_response)


| Injury Description | Injury Severity | Medical Team Involved | Was Play Stopped? | Injury Location | Did the Player Leave the Field? |
|--------------------|------------------|------------------------|--------------------|------------------|-------------------------------|
| The player fell after a heavy tackle from the opponent and showed signs of pain in his left knee. The medical team quickly came to assess the injury. | Severe         | Yes                    | Yes                | Left knee       | Yes                            |
